In [95]:
# using pd.set_option()
# to widen the output 
# display
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

# show the dataframe

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt


In [2]:
##Biblioteca em python para conversão de datas
from datetime import datetime, timedelta

In [3]:
from obspy.core import read

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
gagnepian_2006_catalog = pd.read_csv('resource\gagnepian_2006_catalog.csv')

In [5]:
lognonne_2003_catalog = pd.read_csv('resource\lognonne_2003_catalog.csv')

In [6]:
levent_1008weber = pd.read_csv('resource\levent.1008weber.csv')

In [7]:
nakamura_1979_sm_locations = pd.read_csv('resource\\nakamura_1979_sm_locations.csv');

In [107]:
 nakamura_1983_ai_locations1 = pd.read_csv('resource\\nakamura_1983_ai_locations.csv')

In [9]:
nakamura_1983_m_arrivals = pd.read_csv('resource\\nakamura_1983_m_arrivals.csv')

In [10]:
nakamura_1983_sm_arrivals = pd.read_csv('resource\\nakamura_1983_sm_arrivals.csv')

In [11]:
nakamura_2005_dm_arrivals = pd.read_csv('resource\\nakamura_2005_dm_arrivals.csv')

In [12]:
nakamura_2005_dm_locations = pd.read_csv('resource\\nakamura_2005_dm_locations.csv')

In [13]:
weber_2011_dmq_s_picks = pd.read_csv('resource\\weber_2011_dmq_s_picks.csv')

In [108]:
nakamura_1983_ai_locations1

,AI,Lat,Long,Y,JD,Hour,Min,Sec
0,12 LM,-3.94,-21.20,69,324,22,17,17.7
1,13 S-IVB,-2.75,-27.86,70,105,1,9,41.0
2,14 S-IVB,-8.09,-26.02,71,35,7,40,55.4
3,14 LM,-3.42,-19.67,71,38,0,45,26.7
4,15 S-IVB,-1.51,-11.81,71,210,20,58,42.9
5,15 LM,26.36,0.25,71,215,3,3,37.0
6,17 S-IVB,-4.21,-12.31,72,345,20,32,42.3
7,17 LM,19.99,30.51,72,350,6,50,20.8


In [14]:
##Funções para processar os dados 

def julian_to_gregorian(julian_date):
    base_date = datetime(1900, 1, 1)  # Defina a data base para o início do século 20
    delta = timedelta(days=julian_date - 1)
    gregorian_date = base_date + delta
    return gregorian_date

def converter_coluna_juliano_para_gregoriano(df, coluna_juliano, nova_coluna_gregoriano):
    df[nova_coluna_gregoriano] = df[coluna_juliano].apply(julian_to_gregorian)
    return df

def converter_dia_numero_para_dia_mes(df, coluna_dia_numero, atual_coluna_ano, nova_coluna_dia, nova_coluna_mes):
    # Crie uma nova coluna 'data' apenas para os casos em que 'coluna_dia_numero' e 'Ano' são diferentes de zero
    mask = (df[coluna_dia_numero] != 0) & (df[atual_coluna_ano] != 0)
    df.loc[mask, 'data'] = df[mask].apply(lambda row: datetime.strptime(f"{int(row[atual_coluna_ano])}-{int(row[coluna_dia_numero])}", "%Y-%j"), axis=1)
    
    # Extraia o dia e o mês e adicione as novas colunas
    df[nova_coluna_dia] = df['data'].dt.day
    df[nova_coluna_mes] = df['data'].dt.month
    
    # Descarte a coluna temporária 'data'
    df = df.drop(columns=['data'])
    
    return df

##ajustar_formato_ano de dois dígitos para quatro. Exemplo: 72 para 1972
##prefixo é os dois números fixos que vai ser concatenados. Exemplo: 19
def ajustar_formato_ano(df, coluna_ano, prefixo):
    df[coluna_ano] = df[coluna_ano].apply(lambda x: int(f"{prefixo}{x:02d}") if x != 0 else 0)
    return df


def separar_data_hora(df, coluna_original, nova_coluna_ano, nova_coluna_mes, nova_coluna_dia, nova_coluna_hora, nova_coluna_minuto):
    df[coluna_original] = df[coluna_original].astype(str)

    # Extrair componentes
    df[nova_coluna_ano] = df[coluna_original].str[:2].astype(int) + 1900  # Assumindo que YY representa o ano
    df[nova_coluna_mes] = df[coluna_original].str[2:4].astype(int)
    df[nova_coluna_dia] = df[coluna_original].str[4:6].astype(int)
    df[nova_coluna_hora] = df[coluna_original].str[6:8].astype(int)
    df[nova_coluna_minuto] = df[coluna_original].str[8:10].astype(int)

    return df

In [15]:
##Ver o total de linhas de cada dataset em tabela e quantidade de nulos
total = pd.DataFrame({'Name':[
    "gagnepian_2006_catalog","lognonne_2003_catalog",
    "levent_1008weber", "nakamura_1979_sm_locations",
    "nakamura_1983_ai_locations","nakamura_1983_m_arrivals",
    "nakamura_1983_sm_arrivals","nakamura_2005_dm_locations",
    "weber_2011_dmq_s_picks"
],
'Total de linhas no dataset':[
    gagnepian_2006_catalog.count().sum(),
    lognonne_2003_catalog.count().sum(),
    levent_1008weber.count().sum(),
    nakamura_1979_sm_locations.count().sum(),
    nakamura_1983_m_arrivals.count().sum(),
    nakamura_1983_ai_locations.count().sum(),
    nakamura_1983_sm_arrivals.count().sum(),
    nakamura_2005_dm_locations.count().sum(),
    weber_2011_dmq_s_picks.count().sum()],
'Total de nulos nos datasets':[
    gagnepian_2006_catalog.isnull().sum().sum(),
    lognonne_2003_catalog.isnull().sum().sum(),
    levent_1008weber.isnull().sum().sum(),
    nakamura_1979_sm_locations.isnull().sum().sum(),
    nakamura_1983_m_arrivals.isnull().sum().sum(),
    nakamura_1983_ai_locations.isnull().sum().sum(),
    nakamura_1983_sm_arrivals.isnull().sum().sum(),
    nakamura_2005_dm_locations.isnull().sum().sum(),
    weber_2011_dmq_s_picks.isnull().sum().sum()   
]})




In [16]:
nakamura_1983_ai_locations

,AI,Lat,Long,Y,JD,Hour,Min,Sec
0,12 LM,-3.94,-21.20,69,324,22,17,17.7
1,13 S-IVB,-2.75,-27.86,70,105,1,9,41.0
2,14 S-IVB,-8.09,-26.02,71,35,7,40,55.4
3,14 LM,-3.42,-19.67,71,38,0,45,26.7
4,15 S-IVB,-1.51,-11.81,71,210,20,58,42.9
5,15 LM,26.36,0.25,71,215,3,3,37.0
6,17 S-IVB,-4.21,-12.31,72,345,20,32,42.3
7,17 LM,19.99,30.51,72,350,6,50,20.8


In [17]:
total

,Name,Total de linhas no dataset,Total de nulos nos datasets
0,gagnepian_2006_catalog,696,0
1,lognonne_2003_catalog,1291,302
2,levent_1008weber,181447,314719
3,nakamura_1979_sm_locations,224,28
4,nakamura_1983_ai_locations,486,424
5,nakamura_1983_m_arrivals,64,0
6,nakamura_1983_sm_arrivals,403,213
7,nakamura_2005_dm_locations,954,0
8,weber_2011_dmq_s_picks,100,90


In [18]:
#Preencher Year Nan ou null com 0
nakamura_1983_m_arrivals.loc[ nakamura_1983_m_arrivals['Year'].isna() == True, 'Year' ] = 0
nakamura_1983_m_arrivals.loc[ nakamura_1983_m_arrivals['Day'].isna() == True, 'Day' ] = 0
nakamura_1983_m_arrivals.loc[ nakamura_1983_m_arrivals['Hour'].isna() == True, 'Hour' ] = 0

In [19]:
#Converter Year de float para int
nakamura_1983_m_arrivals['Year'] = nakamura_1983_m_arrivals['Year'].astype(int)
nakamura_1983_m_arrivals['Day'] = nakamura_1983_m_arrivals['Day'].astype(int)
nakamura_1983_m_arrivals['Hour'] = nakamura_1983_m_arrivals['Hour'].astype(int)

In [20]:
nakamura_1983_m_arrivals = converter_dia_numero_para_dia_mes( nakamura_1983_m_arrivals, 'Day', 'Year','Processed_day', 'Processed_month' )

In [21]:
##Como todos os registros da Station estão sem dados de datas optamos por excluí-los 

nakamura_1983_m_arrivals = nakamura_1983_m_arrivals[(nakamura_1983_m_arrivals['Station'] != 14) & (nakamura_1983_m_arrivals['Station'] != 15) & (nakamura_1983_m_arrivals['Station'] != 16)]

In [22]:
##Como só sobrou a station 12. Decidimos em não usar nakamura_1983_m_arrivals no projeto.
nakamura_1983_m_arrivals

,Year,Day,Hour,Station,P_YN,P_NG,P_JK,P_PH,P_Mean,S_YN,S_NG,S_JK,S_PH,S_Mean,Processed_day,Processed_month
0,1971,143,2220,12,19.5,NaN,NaN,19.5,19.5,NaN,NaN,NaN,50.0,50.0,23.0,5.0
2,1971,163,1051,12,51.9,NaN,NaN,52.0,52.0,NaN,NaN,NaN,164.0,164.0,12.0,6.0
4,1971,293,1808,12,31.9,NaN,NaN,35.5,33.7,NaN,NaN,NaN,143.0,143.0,20.0,10.0
7,1972,134,847,12,25.2,25.2,25.2,25.0,25.2,NaN,62.7,NaN,55.0,58.9,13.0,5.0
11,1972,199,2156,12,114.0,115.0,114.0,114.5,114.4,NaN,NaN,NaN,NaN,NaN,17.0,7.0
15,1972,213,1808,12,196.1,196.4,196.1,NaN,196.2,NaN,345.5,345.5,NaN,345.5,31.0,7.0
19,1972,324,1824,12,89.8,87.6,NaN,90.0,89.1,NaN,284.2,NaN,286.0,285.1,19.0,11.0
23,1973,113,1355,12,109.0,NaN,NaN,NaN,109.0,NaN,NaN,NaN,NaN,NaN,23.0,4.0
27,1973,262,932,12,33.8,NaN,NaN,NaN,33.8,NaN,NaN,NaN,NaN,NaN,19.0,9.0
31,1974,325,1316,12,162.1,NaN,NaN,149.0,155.6,NaN,NaN,NaN,294.0,294.0,21.0,11.0


In [23]:
#Converter Sec de float para int
nakamura_1983_ai_locations['Sec'] = nakamura_1983_ai_locations['Sec'].astype(int)

In [24]:
nakamura_1983_ai_locations['Y'] = nakamura_1983_ai_locations['Y'].astype(int)

In [25]:
nakamura_1983_ai_locations['Y']

0    69
1    70
2    71
3    71
4    71
5    71
6    72
7    72
Name: Y, dtype: int32

In [26]:
nakamura_1983_ai_locations

,AI,Lat,Long,Y,JD,Hour,Min,Sec
0,12 LM,-3.94,-21.20,69,324,22,17,17
1,13 S-IVB,-2.75,-27.86,70,105,1,9,41
2,14 S-IVB,-8.09,-26.02,71,35,7,40,55
3,14 LM,-3.42,-19.67,71,38,0,45,26
4,15 S-IVB,-1.51,-11.81,71,210,20,58,42
5,15 LM,26.36,0.25,71,215,3,3,37
6,17 S-IVB,-4.21,-12.31,72,345,20,32,42
7,17 LM,19.99,30.51,72,350,6,50,20


In [27]:
##ajustar_formato_ano
nakamura_1983_ai_locations['Y'] = ajustar_formato_ano(nakamura_1983_ai_locations, 'Y', '19')

ValueError: Columns must be same length as key

In [28]:
#df, coluna_dia_numero, nova_coluna_dia, nova_coluna_mes
nakamura_1983_ai_locations = converter_dia_numero_para_dia_mes(nakamura_1983_ai_locations,'JD', 'Y', 'Processed_day','Processed_month')

<ipython-input-14-5af5ca000d42>:16: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[mask, 'data'] = df[mask].apply(lambda row: datetime.strptime(f"{int(row[atual_coluna_ano])}-{int(row[coluna_dia_numero])}", "%Y-%j"), axis=1)


In [29]:
nakamura_1983_ai_locations[['Y','Processed_month','Processed_day','Hour','Min','Sec','Lat','Long','AI']]

,Y,Processed_month,Processed_day,Hour,Min,Sec,Lat,Long,AI
0,1969,11,20,22,17,17,-3.94,-21.20,12 LM
1,1970,4,15,1,9,41,-2.75,-27.86,13 S-IVB
2,1971,2,4,7,40,55,-8.09,-26.02,14 S-IVB
3,1971,2,7,0,45,26,-3.42,-19.67,14 LM
4,1971,7,29,20,58,42,-1.51,-11.81,15 S-IVB
5,1971,8,3,3,3,37,26.36,0.25,15 LM
6,1972,12,10,20,32,42,-4.21,-12.31,17 S-IVB
7,1972,12,15,6,50,20,19.99,30.51,17 LM


In [30]:
nakamura_1983_sm_arrivals[( nakamura_1983_sm_arrivals['Year'].isna() ) & ( nakamura_1983_sm_arrivals['Day'].isna() )]

,Year,Day,Hour,Station,P_YN,P_NG,P_JK,P_PH,P_Mean,S_YN,S_NG,S_JK,S_PH,S_Mean
1,NaN,NaN,NaN,14,1.9,NaN,NaN,1.5,1.7,182.0,NaN,NaN,192.0,187.0
3,NaN,NaN,NaN,14,60.2,NaN,NaN,60.0,60.1,184.0,NaN,NaN,186.0,185.0
5,NaN,NaN,NaN,14,70.4,NaN,NaN,72.5,71.5,225.0,NaN,NaN,233.0,229.0
7,NaN,NaN,NaN,15,52.0,NaN,NaN,NaN,52.0,250.0,NaN,NaN,NaN,250.0
9,NaN,NaN,NaN,14,35.5,35.9,35.5,35.0,35.5,280.0,NaN,272.5,NaN,276.3
10,NaN,NaN,NaN,15,100.0,99.7,100.0,109.0,102.2,465.0,NaN,398.5,NaN,431.8
11,NaN,NaN,NaN,16,27.2,27.8,27.9,27.5,27.6,270.0,259.4,258.6,266.0,263.5
13,NaN,NaN,NaN,14,66.6,66.5,67.0,54.0,63.5,219.0,NaN,207.5,215.0,213.8
14,NaN,NaN,NaN,15,147.5,145.7,147.5,156.0,149.2,360.0,NaN,346.0,NaN,353.0
15,NaN,NaN,NaN,16,190.7,198.5,191.0,NaN,193.4,430.0,412.5,422.5,NaN,421.7


In [31]:
##
nakamura_1983_sm_arrivals[(~ nakamura_1983_sm_arrivals['Year'].isna() ) & (~ nakamura_1983_sm_arrivals['Day'].isna() )]

,Year,Day,Hour,Station,P_YN,P_NG,P_JK,P_PH,P_Mean,S_YN,S_NG,S_JK,S_PH,S_Mean
0,1971.0,107.0,705.0,12,9.6,NaN,NaN,10.0,9.8,212.0,NaN,NaN,210.0,211.0
2,1971.0,140.0,1728.0,12,NaN,NaN,NaN,NaN,NaN,183.0,NaN,NaN,183.5,183.3
4,1971.0,192.0,1327.0,12,58.2,NaN,NaN,54.3,56.2,200.0,NaN,NaN,200.0,200.0
6,1972.0,2.0,2233.0,14,170.0,NaN,NaN,NaN,170.0,485.0,NaN,NaN,NaN,485.0
8,1973.0,72.0,801.0,12,34.2,34.1,34.2,34.0,34.1,267.0,272.0,267.0,NaN,268.7
12,1973.0,171.0,2024.0,12,56.1,55.0,56.0,NaN,55.7,180.0,185.3,187.5,182.0,183.7
16,1974.0,192.0,50.0,12,138.7,138.5,138.7,NaN,138.6,459.0,NaN,417.0,NaN,438.0
20,1975.0,3.0,146.0,12,33.5,33.6,33.5,34.0,33.7,NaN,269.0,255.0,NaN,262.0
24,1975.0,12.0,317.0,14,137.7,NaN,137.5,140.0,138.4,368.0,NaN,356.0,NaN,362.0
27,1975.0,44.0,2204.0,14,58.3,58.2,58.0,NaN,58.2,108.0,107.8,108.0,NaN,107.9


In [32]:
##Os dados desta camada estão coprometido. Precisa verificar a relevância já que há tantos nulos: nakamura_1983_sm_arrivals

In [33]:
nakamura_2005_dm_arrivals

,A,12P,14P,15P,16P,12S,14S,15S,16S
0,1,138,140,197,184,230,233,341,323
1,2,,,,,220,228,,
2,3,,122,,,227,230,311,330
3,4,,,,,205,203,,270
4,5,,141,,,232,242,,351
...,...,...,...,...,...,...,...,...,...
161,286,,,118,,,,248,321
162,287,,,,,,330,246,272
163,288,,,,,,185,,280
164,289,,,,,,277,239,


In [34]:
nakamura_2005_dm_arrivals[( nakamura_2005_dm_arrivals['16S'] != ' ' )]

,A,12P,14P,15P,16P,12S,14S,15S,16S
0,1,138,140,197,184,230,233,341,323
2,3,,122,,,227,230,311,330
3,4,,,,,205,203,,270
4,5,,141,,,232,242,,351
5,6,190,188,120,155,373,360,245,307
...,...,...,...,...,...,...,...,...,...
160,285,171,165,,89,,,242,285
161,286,,,118,,,,248,321
162,287,,,,,,330,246,272
163,288,,,,,,185,,280


In [35]:
##Os dados desta camada estão coprometido. Precisa verificar a relevância já que há tantos nulos: nakamura_2005_dm_arrivals

In [36]:
nakamura_2005_dm_locations

,A,Side,Lat,Lat_err,Long,Long_err,Depth,Depth_err,Assumed
0,1,N,-15.7,2.4,-36.6,4.6,867,29,N
1,3,N,-2.9,1.7,-50.3,6.3,946,22,N
2,5,N,1.1,94.2,-44.7,16.4,933,109,Y
3,6,N,43.5,2.9,55.5,9.5,844,33,N
4,7,N,25.0,1.7,53.2,8.0,893,27,N
...,...,...,...,...,...,...,...,...,...
101,283,N,7.1,1.2,22.5,4.7,984,76,N
102,285,F,42.9,3.7,110.9,14.2,933,109,Y
103,286,N,54.2,6.5,56.5,14.0,933,109,Y
104,287,N,24.8,5.7,35.7,13.5,933,109,Y


In [37]:
## vou juntar Nakamura_2005_DM_arrivals e nakamura_2005_dm_locations com  e ver se consigo algum resultado

In [38]:
nakamura_2005_dm_locations_arrivals = pd.concat([nakamura_2005_dm_locations, nakamura_2005_dm_arrivals], axis=1)

In [39]:
nakamura_2005_dm_locations_arrivals

,A,Side,Lat,Lat_err,Long,Long_err,Depth,Depth_err,Assumed,A,12P,14P,15P,16P,12S,14S,15S,16S
0,1.0,N,-15.7,2.4,-36.6,4.6,867.0,29.0,N,1,138,140,197,184,230,233,341,323
1,3.0,N,-2.9,1.7,-50.3,6.3,946.0,22.0,N,2,,,,,220,228,,
2,5.0,N,1.1,94.2,-44.7,16.4,933.0,109.0,Y,3,,122,,,227,230,311,330
3,6.0,N,43.5,2.9,55.5,9.5,844.0,33.0,N,4,,,,,205,203,,270
4,7.0,N,25.0,1.7,53.2,8.0,893.0,27.0,N,5,,141,,,232,242,,351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286,,,118,,,,248,321
162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287,,,,,,330,246,272
163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288,,,,,,185,,280
164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,289,,,,,,277,239,


In [40]:
nakamura_1979_sm_locations.drop(columns=['Day'])

,Year,H,M,S,Lat,Long,Magnitude,Comments
0,1971,7,0,55,48.0,35.0,2.8,NaN
1,1971,17,25,10,42.0,-24.0,2.0,NaN
2,1971,13,24,45,43.0,-47.0,1.9,Or lat -42 long -60
3,1972,22,29,40,54.0,101.0,1.9,NaN
4,1972,14,35,55,12.0,46.0,1.0,NaN
5,1972,23,8,20,51.0,45.0,1.4,NaN
6,1972,3,50,15,-20.0,-80.0,1.2,NaN
7,1973,22,52,10,33.0,35.0,0.8,NaN
8,1973,7,56,30,-84.0,-134.0,3.2,NaN
9,1973,20,22,0,-1.0,-71.0,2.2,NaN


In [41]:
nakamura_1979_sm_locations = converter_dia_numero_para_dia_mes(nakamura_1979_sm_locations, 'Day', 'Year', 'Processed_day', 'Processed_month')

<ipython-input-14-5af5ca000d42>:16: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[mask, 'data'] = df[mask].apply(lambda row: datetime.strptime(f"{int(row[atual_coluna_ano])}-{int(row[coluna_dia_numero])}", "%Y-%j"), axis=1)


In [42]:
gagnepian_2006_catalog = separar_data_hora( gagnepian_2006_catalog, 'Date','Year','Processed_month','Processed_day', 'Hour','Minute')

In [43]:
gagnepian_2006_catalog[gagnepian_2006_catalog['Year'].isna() == True]

,Type,Lat,Long,Depth,Date,YN_Lat,YN_Lon,YN_Depth,Phi,Delta-a,Delta-b,Depth_err,Year,Processed_month,Processed_day,Hour,Minute


In [44]:
gagnepian_2006_catalog.isna().sum()

Type               0
Lat                0
Long               0
Depth              0
Date               0
YN_Lat             0
YN_Lon             0
YN_Depth           0
Phi                0
Delta-a            0
Delta-b            0
Depth_err          0
Year               0
Processed_month    0
Processed_day      0
Hour               0
Minute             0
dtype: int64

In [45]:
## planilha gagnepian_2006_catalog está completa. Podendo utilizar nos estudos

In [46]:
lognonne_2003_catalog = separar_data_hora( lognonne_2003_catalog, 'Date','Year','Processed_month','Processed_day', 'Hour','Minute')

In [47]:
lognonne_2003_catalog.loc[lognonne_2003_catalog['Depth'].isna() == True, 'Depth'] = 0

In [48]:
lognonne_2003_catalog

,Type,Lat,Long,Depth,Phi,Delta-a,Delta-b,Depth_err,Date,Seconds,...,14S_EC,15S,15S_EC,16S,16S_EC,Year,Processed_month,Processed_day,Hour,Minute
0,12 LM,-3.94,-21.20,0.0,0,0.0,0.0,0,6911202217,17.70,...,NaN,NaN,NaN,NaN,NaN,1969,11,20,22,17
1,13 S-IVB,-2.75,-27.86,0.0,0,0.0,0.0,0,7004150209,41.00,...,NaN,NaN,NaN,NaN,NaN,1970,4,15,2,9
2,14 S-IVB,-8.09,-26.02,0.0,0,0.0,0.0,0,7102040740,55.40,...,NaN,NaN,NaN,NaN,NaN,1971,2,4,7,40
3,14 LM,-3.42,-19.67,0.0,0,0.0,0.0,0,7102070045,25.70,...,NaN,NaN,NaN,NaN,NaN,1971,2,7,0,45
4,15 S-IVB,-1.51,-11.81,0.0,0,0.0,0.0,0,7107292058,42.90,...,NaN,NaN,NaN,NaN,NaN,1971,7,29,20,58
5,15 LM,26.36,0.25,0.0,0,0.0,0.0,0,7108030303,37.00,...,NaN,NaN,NaN,NaN,NaN,1971,8,3,3,3
6,16 S-IVB,1.30,-23.80,0.0,0,0.0,0.0,0,7204192102,4.00,...,NaN,NaN,NaN,NaN,NaN,1972,4,19,21,2
7,17 S-IVB,-4.21,-12.31,0.0,0,0.0,0.0,0,7212102032,42.30,...,NaN,NaN,NaN,NaN,NaN,1972,12,10,20,32
8,M,74.10,2.60,0.0,16,2.7,1.1,0,7201040631,19.72,...,NaN,350.0,1.0,NaN,NaN,1972,1,4,6,31
9,M,1.50,-17.10,0.0,-52,0.3,0.2,0,7205130845,39.49,...,2.0,NaN,NaN,NaN,NaN,1972,5,13,8,45


In [49]:
weber_2011_dmq_s_picks

##campos com nenhuma descrição e sem sentido sem um estudo mais aprofundado

,A#,S12,S14,S15,S16
0,A1,226.0,235.7,338.9,314.6
1,A3,255.4,NaN,NaN,NaN
2,A5,235.1,NaN,NaN,NaN
3,A6,418.4,NaN,298.0,371.3
4,A7,410.6,409.3,366.8,360.9
5,A8,NaN,NaN,NaN,328.8
6,A9,244.1,NaN,NaN,NaN
7,A10,NaN,289.0,NaN,297.2
8,A13,260.5,NaN,NaN,NaN
9,A14,268.7,NaN,NaN,NaN


In [50]:
## N]ao consta latitudae, çpngitude, nem magnitude e nem Profundidade. Parece que tem dados de quadrante mas preciso de mais tempo para pesquisar o estudo. Além de ter muitos nulos.
##levent_1008weber

In [51]:
##Renomear as colunas para ficarem do modo padrao

nakamura_1983_ai_locations.rename(columns={'Y':'Year'}, inplace=True)
nakamura_1983_ai_locations.rename(columns={'Processed_day':'Day'}, inplace=True)
nakamura_1983_ai_locations.rename(columns={'Processed_month':'Month'}, inplace=True)

In [52]:
##ordena-las para ficarem do modo padrao

nakamura_1983_ai_locations = nakamura_1983_ai_locations[['Year','Month','Day','Hour','Min','Sec','Lat','Long','AI']]

In [53]:
nakamura_1983_ai_locations

,Year,Month,Day,Hour,Min,Sec,Lat,Long,AI
0,1969,11,20,22,17,17,-3.94,-21.20,12 LM
1,1970,4,15,1,9,41,-2.75,-27.86,13 S-IVB
2,1971,2,4,7,40,55,-8.09,-26.02,14 S-IVB
3,1971,2,7,0,45,26,-3.42,-19.67,14 LM
4,1971,7,29,20,58,42,-1.51,-11.81,15 S-IVB
5,1971,8,3,3,3,37,26.36,0.25,15 LM
6,1972,12,10,20,32,42,-4.21,-12.31,17 S-IVB
7,1972,12,15,6,50,20,19.99,30.51,17 LM


In [54]:
##Renomear as colunas para ficarem do modo padrao

lognonne_2003_catalog.rename(columns={'Processed_day':'Day'}, inplace=True)
lognonne_2003_catalog.rename(columns={'Processed_month':'Month'}, inplace=True)

In [55]:
lognonne_2003_catalog = lognonne_2003_catalog[['Year','Month','Day','Hour','Minute','Lat','Long','Depth']]

In [56]:
nakamura_1979_sm_locations = nakamura_1979_sm_locations.drop(columns=['Day'])

In [57]:
##Renomear as colunas para ficarem do modo padrao

nakamura_1979_sm_locations.rename(columns={'H':'Hour'}, inplace=True)
nakamura_1979_sm_locations.rename(columns={'M':'Minute'}, inplace=True)
nakamura_1979_sm_locations.rename(columns={'S':'Sec'}, inplace=True)
nakamura_1979_sm_locations.rename(columns={'Processed_day':'Day'}, inplace=True)
nakamura_1979_sm_locations.rename(columns={'Processed_month':'Month'}, inplace=True)

In [58]:
nakamura_1979_sm_locations = nakamura_1979_sm_locations[['Year','Month','Day','Hour','Minute','Sec','Lat','Long','Magnitude','Comments']] 

In [59]:
##Renomear as colunas para ficarem do modo padrao

gagnepian_2006_catalog.rename(columns={'Minute':'Min'}, inplace=True)
gagnepian_2006_catalog.rename(columns={'Processed_day':'Day'}, inplace=True)
gagnepian_2006_catalog.rename(columns={'Processed_month':'Month'}, inplace=True)

In [60]:
gagnepian_2006_catalog = gagnepian_2006_catalog[['Year','Month','Day','Hour','Min','Lat','Long','Depth']] 

In [61]:
##Renomear as colunas para ficarem do modo padrao

lognonne_2003_catalog.rename(columns={'Minute':'Min'}, inplace=True)

<ipython-input-61-af89b3ad5937>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lognonne_2003_catalog.rename(columns={'Minute':'Min'}, inplace=True)


In [62]:
nakamura_1979_sm_locations   

,Year,Month,Day,Hour,Minute,Sec,Lat,Long,Magnitude,Comments
0,1971,4,17,7,0,55,48.0,35.0,2.8,NaN
1,1971,5,20,17,25,10,42.0,-24.0,2.0,NaN
2,1971,7,11,13,24,45,43.0,-47.0,1.9,Or lat -42 long -60
3,1972,1,2,22,29,40,54.0,101.0,1.9,NaN
4,1972,9,17,14,35,55,12.0,46.0,1.0,NaN
5,1972,12,6,23,8,20,51.0,45.0,1.4,NaN
6,1972,12,9,3,50,15,-20.0,-80.0,1.2,NaN
7,1973,2,8,22,52,10,33.0,35.0,0.8,NaN
8,1973,3,13,7,56,30,-84.0,-134.0,3.2,NaN
9,1973,6,20,20,22,0,-1.0,-71.0,2.2,NaN


In [63]:
## Adicionar campo label conforme a descrição abaixo:
## nakamura_1979_sm_locations = nakamura(1979)
## lognonne_2003_catalog = lognonne(2003)
## nakamura_1983_ai_locations = nakamura(1983) = Tipo 1

##nakamura_1979_sm_locations['label'] = 'nakamura(1979)'
##lognonne_2003_catalog['label'] = 'lognonne(2003)'
##nakamura_1983_ai_locations['label'] = 'nakamura(1983)'





In [64]:
def compose_ai(value):
    name = ' Apollo '.join(value.split()[::-1])
    return name[-10::]

nakamura_1983_ai_locations['label'] = nakamura_1983_ai_locations['AI'].apply(compose_ai)

nakamura_1983_ai_locations

,Year,Month,Day,Hour,Min,Sec,Lat,Long,AI,label
0,1969,11,20,22,17,17,-3.94,-21.20,12 LM,Apollo 12
1,1970,4,15,1,9,41,-2.75,-27.86,13 S-IVB,Apollo 13
2,1971,2,4,7,40,55,-8.09,-26.02,14 S-IVB,Apollo 14
3,1971,2,7,0,45,26,-3.42,-19.67,14 LM,Apollo 14
4,1971,7,29,20,58,42,-1.51,-11.81,15 S-IVB,Apollo 15
5,1971,8,3,3,3,37,26.36,0.25,15 LM,Apollo 15
6,1972,12,10,20,32,42,-4.21,-12.31,17 S-IVB,Apollo 17
7,1972,12,15,6,50,20,19.99,30.51,17 LM,Apollo 17


In [65]:
agrupados_planilhas = [nakamura_1979_sm_locations,lognonne_2003_catalog,nakamura_1983_ai_locations]

In [66]:
nakamura_1983_ai_locations

,Year,Month,Day,Hour,Min,Sec,Lat,Long,AI,label
0,1969,11,20,22,17,17,-3.94,-21.20,12 LM,Apollo 12
1,1970,4,15,1,9,41,-2.75,-27.86,13 S-IVB,Apollo 13
2,1971,2,4,7,40,55,-8.09,-26.02,14 S-IVB,Apollo 14
3,1971,2,7,0,45,26,-3.42,-19.67,14 LM,Apollo 14
4,1971,7,29,20,58,42,-1.51,-11.81,15 S-IVB,Apollo 15
5,1971,8,3,3,3,37,26.36,0.25,15 LM,Apollo 15
6,1972,12,10,20,32,42,-4.21,-12.31,17 S-IVB,Apollo 17
7,1972,12,15,6,50,20,19.99,30.51,17 LM,Apollo 17


In [67]:
agregado = pd.concat(agrupados_planilhas)

In [68]:
agregado = agregado[['Year','Month','Day','Hour','Min','Sec','Lat','Long','Depth', 'Magnitude','AI','label','Comments']] 

In [69]:
##Conferindo se há linhas duplicadas e está tudo ok.
temp = agregado.drop_duplicates()
temp

,Year,Month,Day,Hour,Min,Sec,Lat,Long,Depth,Magnitude,AI,label,Comments
0,1971,4,17,7,NaN,55.0,48.00,35.00,NaN,2.8,NaN,NaN,NaN
1,1971,5,20,17,NaN,10.0,42.00,-24.00,NaN,2.0,NaN,NaN,NaN
2,1971,7,11,13,NaN,45.0,43.00,-47.00,NaN,1.9,NaN,NaN,Or lat -42 long -60
3,1972,1,2,22,NaN,40.0,54.00,101.00,NaN,1.9,NaN,NaN,NaN
4,1972,9,17,14,NaN,55.0,12.00,46.00,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1971,2,7,0,45.0,26.0,-3.42,-19.67,NaN,NaN,14 LM,Apollo 14,NaN
4,1971,7,29,20,58.0,42.0,-1.51,-11.81,NaN,NaN,15 S-IVB,Apollo 15,NaN
5,1971,8,3,3,3.0,37.0,26.36,0.25,NaN,NaN,15 LM,Apollo 15,NaN
6,1972,12,10,20,32.0,42.0,-4.21,-12.31,NaN,NaN,17 S-IVB,Apollo 17,NaN


In [70]:
##Renomear campo Long para lng, Magnitude para magnitude, Lat para lat
agregado.rename(columns={'Magnitude':'magnitude'}, inplace=True)
agregado.rename(columns={'Long':'lng'}, inplace=True)
agregado.rename(columns={'Lat':'lat'}, inplace=True)
agregado

,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments
0,1971,4,17,7,NaN,55.0,48.00,35.00,NaN,2.8,NaN,NaN,NaN
1,1971,5,20,17,NaN,10.0,42.00,-24.00,NaN,2.0,NaN,NaN,NaN
2,1971,7,11,13,NaN,45.0,43.00,-47.00,NaN,1.9,NaN,NaN,Or lat -42 long -60
3,1972,1,2,22,NaN,40.0,54.00,101.00,NaN,1.9,NaN,NaN,NaN
4,1972,9,17,14,NaN,55.0,12.00,46.00,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1971,2,7,0,45.0,26.0,-3.42,-19.67,NaN,NaN,14 LM,Apollo 14,NaN
4,1971,7,29,20,58.0,42.0,-1.51,-11.81,NaN,NaN,15 S-IVB,Apollo 15,NaN
5,1971,8,3,3,3.0,37.0,26.36,0.25,NaN,NaN,15 LM,Apollo 15,NaN
6,1972,12,10,20,32.0,42.0,-4.21,-12.31,NaN,NaN,17 S-IVB,Apollo 17,NaN


In [71]:
##Setar valor 0 onde estiver null, nan ou vazio
agregado.loc[agregado['Min'].isna() == True,'Min'] = 0
agregado.loc[agregado['Sec'].isna() == True,'Sec'] = 0
agregado.loc[agregado['Comments'].isna() == True,'Comments'] = ''
agregado

,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments
0,1971,4,17,7,0.0,55.0,48.00,35.00,NaN,2.8,NaN,NaN,
1,1971,5,20,17,0.0,10.0,42.00,-24.00,NaN,2.0,NaN,NaN,
2,1971,7,11,13,0.0,45.0,43.00,-47.00,NaN,1.9,NaN,NaN,Or lat -42 long -60
3,1972,1,2,22,0.0,40.0,54.00,101.00,NaN,1.9,NaN,NaN,
4,1972,9,17,14,0.0,55.0,12.00,46.00,NaN,1.0,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1971,2,7,0,45.0,26.0,-3.42,-19.67,NaN,NaN,14 LM,Apollo 14,
4,1971,7,29,20,58.0,42.0,-1.51,-11.81,NaN,NaN,15 S-IVB,Apollo 15,
5,1971,8,3,3,3.0,37.0,26.36,0.25,NaN,NaN,15 LM,Apollo 15,
6,1972,12,10,20,32.0,42.0,-4.21,-12.31,NaN,NaN,17 S-IVB,Apollo 17,


In [72]:
##Converter campo float em inteiro. Exemplo: 0.1 para 1
agregado['Min'] = agregado['Min'].astype(int)
agregado['Sec'] = agregado['Sec'].astype(int)

In [73]:
"""
def juntar_campos_data_hora(df, coluna_ano, coluna_mes, coluna_dia, coluna_hora, coluna_minuto, coluna_segundo, nova_coluna_data_hora):
    df['Datetime'] = pd.to_datetime(df[[coluna_ano, coluna_mes, coluna_dia, coluna_hora, coluna_minuto, coluna_segundo]].astype(str).agg('-'.join, axis=1))
    df[nova_coluna_data_hora] = df['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df = df.drop(columns=['Datetime'])
    """

"\ndef juntar_campos_data_hora(df, coluna_ano, coluna_mes, coluna_dia, coluna_hora, coluna_minuto, coluna_segundo, nova_coluna_data_hora):\n    df['Datetime'] = pd.to_datetime(df[[coluna_ano, coluna_mes, coluna_dia, coluna_hora, coluna_minuto, coluna_segundo]].astype(str).agg('-'.join, axis=1))\n    df[nova_coluna_data_hora] = df['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')\n    df = df.drop(columns=['Datetime'])\n    "

In [74]:
##Adicionar zero à esquerda dos campos Min e Sec
#agregado.zfill(1 + )

In [75]:
##concatenar os campos datetime em um único date
#agregado['date'] = juntar_campos_data_hora(agregado,'Year','Month','Day','Hour','Min','Sec','date')


In [76]:
agregado.loc[agregado['AI'].isna() != True, 'type']  = 1
agregado.loc[ ( ( agregado['Depth'].isna() != True ) & ( agregado['Depth'] > 0 ) ) , 'type']  = 3
agregado.loc[ ( ( agregado['magnitude'].isna() != True ) & ( agregado['magnitude'] > 0 ) ) , 'type']  = 2


In [77]:
##gerar csv para exportar na pasta raiz do projeto
##agregado.to_csv('dados_agregados.csv')

In [78]:
## Criar a coluna size

In [79]:
## Regra quando for type = 1 inserir o valor 3 em size
agregado.loc[ agregado['type'] == 1,'size' ] = 3

In [80]:
##
agregado.loc[agregado['size'].isna() == True, 'size'] = 0
agregado['size'].astype(int)

0    0
1    0
2    0
3    0
4    0
    ..
3    3
4    3
5    3
6    3
7    3
Name: size, Length: 95, dtype: int32

In [81]:
agregado

,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments,type,size
0,1971,4,17,7,0,55,48.00,35.00,NaN,2.8,NaN,NaN,,2.0,0.0
1,1971,5,20,17,0,10,42.00,-24.00,NaN,2.0,NaN,NaN,,2.0,0.0
2,1971,7,11,13,0,45,43.00,-47.00,NaN,1.9,NaN,NaN,Or lat -42 long -60,2.0,0.0
3,1972,1,2,22,0,40,54.00,101.00,NaN,1.9,NaN,NaN,,2.0,0.0
4,1972,9,17,14,0,55,12.00,46.00,NaN,1.0,NaN,NaN,,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1971,2,7,0,45,26,-3.42,-19.67,NaN,NaN,14 LM,Apollo 14,,1.0,3.0
4,1971,7,29,20,58,42,-1.51,-11.81,NaN,NaN,15 S-IVB,Apollo 15,,1.0,3.0
5,1971,8,3,3,3,37,26.36,0.25,NaN,NaN,15 LM,Apollo 15,,1.0,3.0
6,1972,12,10,20,32,42,-4.21,-12.31,NaN,NaN,17 S-IVB,Apollo 17,,1.0,3.0


In [82]:
## Parte final do trabalho. Não processar após esta célula ##

In [83]:
agregado.sort_values(by=['Year','Month','Day','Hour','Min','Sec','magnitude','Depth'])

,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments,type,size
0,1969,11,20,22,17,0,-3.94,-21.20,0.0,NaN,NaN,NaN,,NaN,0.0
0,1969,11,20,22,17,17,-3.94,-21.20,NaN,NaN,12 LM,Apollo 12,,1.0,3.0
1,1970,4,15,1,9,41,-2.75,-27.86,NaN,NaN,13 S-IVB,Apollo 13,,1.0,3.0
1,1970,4,15,2,9,0,-2.75,-27.86,0.0,NaN,NaN,NaN,,NaN,0.0
2,1971,2,4,7,40,0,-8.09,-26.02,0.0,NaN,NaN,NaN,,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,1977,5,19,6,8,0,-3.40,18.70,1000.0,NaN,NaN,NaN,,3.0,0.0
47,1977,6,9,20,15,0,34.40,59.30,898.0,NaN,NaN,NaN,,3.0,0.0
46,1977,6,12,18,17,0,-36.80,-38.90,980.0,NaN,NaN,NaN,,3.0,0.0
40,1977,6,18,5,1,0,9.30,17.50,1200.0,NaN,NaN,NaN,,3.0,0.0


In [86]:
##Criar um novo índice - index
agregado.reset_index( inplace=True)
agregado

,index,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments,type,size
0,0,1971,4,17,7,0,55,48.00,35.00,NaN,2.8,NaN,NaN,,2.0,0.0
1,1,1971,5,20,17,0,10,42.00,-24.00,NaN,2.0,NaN,NaN,,2.0,0.0
2,2,1971,7,11,13,0,45,43.00,-47.00,NaN,1.9,NaN,NaN,Or lat -42 long -60,2.0,0.0
3,3,1972,1,2,22,0,40,54.00,101.00,NaN,1.9,NaN,NaN,,2.0,0.0
4,4,1972,9,17,14,0,55,12.00,46.00,NaN,1.0,NaN,NaN,,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,90,1971,2,7,0,45,26,-3.42,-19.67,NaN,NaN,14 LM,Apollo 14,,1.0,3.0
91,91,1971,7,29,20,58,42,-1.51,-11.81,NaN,NaN,15 S-IVB,Apollo 15,,1.0,3.0
92,92,1971,8,3,3,3,37,26.36,0.25,NaN,NaN,15 LM,Apollo 15,,1.0,3.0
93,93,1972,12,10,20,32,42,-4.21,-12.31,NaN,NaN,17 S-IVB,Apollo 17,,1.0,3.0


In [87]:
##Renomear índice de index para id_event
agregado.rename(columns={'index':'id_event'}, inplace=True)
agregado

,id_event,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments,type,size
0,0,1971,4,17,7,0,55,48.00,35.00,NaN,2.8,NaN,NaN,,2.0,0.0
1,1,1971,5,20,17,0,10,42.00,-24.00,NaN,2.0,NaN,NaN,,2.0,0.0
2,2,1971,7,11,13,0,45,43.00,-47.00,NaN,1.9,NaN,NaN,Or lat -42 long -60,2.0,0.0
3,3,1972,1,2,22,0,40,54.00,101.00,NaN,1.9,NaN,NaN,,2.0,0.0
4,4,1972,9,17,14,0,55,12.00,46.00,NaN,1.0,NaN,NaN,,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,90,1971,2,7,0,45,26,-3.42,-19.67,NaN,NaN,14 LM,Apollo 14,,1.0,3.0
91,91,1971,7,29,20,58,42,-1.51,-11.81,NaN,NaN,15 S-IVB,Apollo 15,,1.0,3.0
92,92,1971,8,3,3,3,37,26.36,0.25,NaN,NaN,15 LM,Apollo 15,,1.0,3.0
93,93,1972,12,10,20,32,42,-4.21,-12.31,NaN,NaN,17 S-IVB,Apollo 17,,1.0,3.0


In [ ]:
##agregado.sort_values(by=['Year','Month','Day','Hour','Min','Sec','magnitude','Depth'])

In [ ]:
##eliminar dupicados
##agregado[ (agregado['id_event'] == 0) & (agregado['label'].isna()) ]



In [ ]:
##agregado.drop(0)

In [ ]:
##agregado.sort_values(by=['Year','Month','Day','Hour','Min','Sec','magnitude','Depth'])

In [97]:
agregado.sort_values(by=['Year','Month','Day','Hour','Min','Sec','magnitude','Depth'])

,id_event,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments,type,size
28,28,1969,11,20,22,17,0,-3.94,-21.20,0.0,NaN,NaN,NaN,,NaN,0.0
87,87,1969,11,20,22,17,17,-3.94,-21.20,NaN,NaN,12 LM,Apollo 12,,1.0,3.0
88,88,1970,4,15,1,9,41,-2.75,-27.86,NaN,NaN,13 S-IVB,Apollo 13,,1.0,3.0
29,29,1970,4,15,2,9,0,-2.75,-27.86,0.0,NaN,NaN,NaN,,NaN,0.0
30,30,1971,2,4,7,40,0,-8.09,-26.02,0.0,NaN,NaN,NaN,,NaN,0.0
89,89,1971,2,4,7,40,55,-8.09,-26.02,NaN,NaN,14 S-IVB,Apollo 14,,1.0,3.0
31,31,1971,2,7,0,45,0,-3.42,-19.67,0.0,NaN,NaN,NaN,,NaN,0.0
90,90,1971,2,7,0,45,26,-3.42,-19.67,NaN,NaN,14 LM,Apollo 14,,1.0,3.0
0,0,1971,4,17,7,0,55,48.00,35.00,NaN,2.8,NaN,NaN,,2.0,0.0
1,1,1971,5,20,17,0,10,42.00,-24.00,NaN,2.0,NaN,NaN,,2.0,0.0


In [98]:
agregado = agregado.drop([28,29,30,31,32,33,35,15])

In [99]:
agregado

,id_event,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments,type,size
0,0,1971,4,17,7,0,55,48.00,35.00,NaN,2.8,NaN,NaN,,2.0,0.0
1,1,1971,5,20,17,0,10,42.00,-24.00,NaN,2.0,NaN,NaN,,2.0,0.0
2,2,1971,7,11,13,0,45,43.00,-47.00,NaN,1.9,NaN,NaN,Or lat -42 long -60,2.0,0.0
3,3,1972,1,2,22,0,40,54.00,101.00,NaN,1.9,NaN,NaN,,2.0,0.0
4,4,1972,9,17,14,0,55,12.00,46.00,NaN,1.0,NaN,NaN,,2.0,0.0
5,5,1972,12,6,23,0,20,51.00,45.00,NaN,1.4,NaN,NaN,,2.0,0.0
6,6,1972,12,9,3,0,15,-20.00,-80.00,NaN,1.2,NaN,NaN,,2.0,0.0
7,7,1973,2,8,22,0,10,33.00,35.00,NaN,0.8,NaN,NaN,,2.0,0.0
8,8,1973,3,13,7,0,30,-84.00,-134.00,NaN,3.2,NaN,NaN,,2.0,0.0
9,9,1973,6,20,20,0,0,-1.00,-71.00,NaN,2.2,NaN,NaN,,2.0,0.0


In [104]:
agregado = agregado.sort_values(by=['Year','Month','Day','Hour','Min','Sec','magnitude','Depth'])

In [105]:
agregado

,id_event,Year,Month,Day,Hour,Min,Sec,lat,lng,Depth,magnitude,AI,label,Comments,type,size
87,87,1969,11,20,22,17,17,-3.94,-21.20,NaN,NaN,12 LM,Apollo 12,,1.0,3.0
88,88,1970,4,15,1,9,41,-2.75,-27.86,NaN,NaN,13 S-IVB,Apollo 13,,1.0,3.0
89,89,1971,2,4,7,40,55,-8.09,-26.02,NaN,NaN,14 S-IVB,Apollo 14,,1.0,3.0
90,90,1971,2,7,0,45,26,-3.42,-19.67,NaN,NaN,14 LM,Apollo 14,,1.0,3.0
0,0,1971,4,17,7,0,55,48.00,35.00,NaN,2.8,NaN,NaN,,2.0,0.0
1,1,1971,5,20,17,0,10,42.00,-24.00,NaN,2.0,NaN,NaN,,2.0,0.0
2,2,1971,7,11,13,0,45,43.00,-47.00,NaN,1.9,NaN,NaN,Or lat -42 long -60,2.0,0.0
91,91,1971,7,29,20,58,42,-1.51,-11.81,NaN,NaN,15 S-IVB,Apollo 15,,1.0,3.0
92,92,1971,8,3,3,3,37,26.36,0.25,NaN,NaN,15 LM,Apollo 15,,1.0,3.0
3,3,1972,1,2,22,0,40,54.00,101.00,NaN,1.9,NaN,NaN,,2.0,0.0


In [106]:
agregado.to_json('agregado.json',orient = 'records')